In [1]:
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn import preprocessing

In [13]:
"""
Read in train and test as Pandas DataFrames
"""
df_train = pd.read_csv("train_short.csv")
df_test = pd.read_csv("train_short.csv")

In [14]:
df_train.head()

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256,gap
0,c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.19
1,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.60
2,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.49
3,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.36
4,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.98


In [ ]:
df_test.head()

In [15]:
#store gap values
Y_train = df_train.gap.values
#row where testing examples start
test_idx = df_train.shape[0]
#delete 'Id' column
df_test = df_test.drop(['Id'], axis=1)
#delete 'gap' column
df_train = df_train.drop(['gap'], axis=1)


ValueError: labels ['Id'] not contained in axis

In [7]:
df_train2=df_train.drop(['smiles'],axis=1).head()
# df_train2 = preprocessing.normalize(df_train2, norm='l2')

# print df_train.index[[10]]

scaler = preprocessing.StandardScaler().fit(df_train2)
scaler
df_train_2=scaler.transform(df_train2)                               

In [8]:
print df_train_2

[[-1.22474487  0.          0.         ...,  0.          0.          0.        ]
 [ 0.81649658  0.          0.         ...,  0.          0.          0.        ]
 [ 0.81649658  0.          0.         ...,  0.          0.          0.        ]
 [ 0.81649658  0.          0.         ...,  0.          0.          0.        ]
 [-1.22474487  0.          0.         ...,  0.          0.          0.        ]]


In [9]:
#DataFrame with all train and test examples so we can more easily apply feature engineering on
df_all = pd.concat((df_train, df_test), axis=0)
df_all.head()

NameError: name 'df_test' is not defined

In [10]:
"""
Example Feature Engineering

this calculates the length of each smile string and adds a feature column with those lengths
Note: this is NOT a good feature and will result in a lower score!
"""
#smiles_len = np.vstack(df_all.smiles.astype(str).apply(lambda x: len(x)))
#df_all['smiles_len'] = pd.DataFrame(smiles_len)


'\nExample Feature Engineering\n\nthis calculates the length of each smile string and adds a feature column with those lengths\nNote: this is NOT a good feature and will result in a lower score!\n'

In [11]:
#Drop the 'smiles' column
df_all = df_all.drop(['smiles'], axis=1)
vals = df_all.values
X_train = vals[:test_idx]
X_test = vals[test_idx:]
print "Train features:", X_train.shape
print "Train gap:", Y_train.shape
print "Test features:", X_test.shape

Train features: (1000000, 256)
Train gap: (1000000,)
Test features: (824230, 256)


In [22]:
LR = LinearRegression()
BT=time.time()
LR.fit(X_train, Y_train)
ET=time.time()
LR_pred = LR.predict(X_test)

print ET-BT

21.2992088795


In [21]:
RF = RandomForestRegressor()
BT=time.time()
RF.fit(X_train, Y_train)
ET=time.time()
RF_pred = RF.predict(X_test)

print ET-BT

34.5866570473


In [14]:
def write_to_file(filename, predictions):
    with open(filename, "w") as f:
        f.write("Id,Prediction\n")
        for i,p in enumerate(predictions):
            f.write(str(i+1) + "," + str(p) + "\n")

In [15]:
write_to_file("sample1.csv", LR_pred)
write_to_file("sample2.csv", RF_pred)